In [ ]:
from CLS_ring import CLSforRing
from utils import load_and_resample, extract_sec, preprocess, forpred
import torch
import ring_voice
from prediction_denoise import prediction
device = torch.device('cuda:1')
model = ring_voice.model
model.load_state_dict(torch.load("/home/ubuntu/jupyter/SKT/log/ring_voice_binary_2/model.pt"))
model.to(device)
model.eval()

# 코덱별 샘플링 레이트 및 주파수 범위 설정
codec_config = {
    'EVS': {'sample_rate': 16000, 'freq_range': (0, 8000)},
    'AMR-WB': {'sample_rate': 16000, 'freq_range': (0, 8000)},
    'AMR': {'sample_rate': 8000, 'freq_range': (0, 4000)},
}
selected_codec = 'EVS'
config = codec_config[selected_codec]
target_sr = config['sample_rate']
n_fft = 1024  # FFT 크기
hop_length = n_fft // 4  # 일반적으로 hop_length는 n_fft의 1/4로 설정
hann_window = torch.hann_window(n_fft)

weights_path = '/home/ubuntu/jupyter/SKT/submit/weights'
name_model = 'model_unet'
        

: 

In [2]:
import os
import numpy as np
import librosa

test_sample1_path = "/home/ubuntu/jupyter/SKT/data/Scenario/music.wav"
test_sample2_path = "/home/ubuntu/jupyter/SKT/data/Scenario/system_o.wav" # PCM_30_wb
test_sample3_path = "/home/ubuntu/jupyter/SKT/data/Scenario/system_x1.wav" # PCM_15_wb

test_sample4_path = "/home/ubuntu/jupyter/SKT/data/Scenario/music+speech.wav"
test_sample5_path = "/home/ubuntu/jupyter/SKT/data/Scenario/music+speech(noise).wav"

test_sample6_path = "/home/ubuntu/jupyter/SKT/data/Scenario/system_o+speech.wav" # ANNC_1508_wb + start @ 60sec
test_sample7_path = "/home/ubuntu/jupyter/SKT/data/Scenario/system_o+speech(noise).wav" # INTL_TERM_ANM_wb + start @ 4sec


In [ ]:
for path in [test_sample1_path, test_sample2_path, test_sample3_path, test_sample4_path, test_sample5_path, test_sample6_path, test_sample7_path]:
    print("\n",path)
    sample = load_and_resample(path, target_sr)
    RING = CLSforRing('RF')
    for s in range(0, sample.shape[-1]- target_sr, target_sr):
        sample_sec = extract_sec(sample, target_sr, start = int(s/target_sr))
        sample_filter = preprocess(sample_sec, n_fft, hop_length, hann_window, target_sr, config['freq_range']).float()
        is_sys = RING.cls(sample_filter.reshape(1,-1))
        if is_sys == 'no sys':
            with torch.no_grad():
                out_dict = model(forpred(sample_filter.unsqueeze(0)).unsqueeze(0).type(torch.FloatTensor).transpose(2,3).to(device))
            test_pred=out_dict['event_logit']
            no_speech = torch.argmax(test_pred, dim = -1).item()
            if no_speech:
                print(f"{int(s/target_sr)} : {int(s/target_sr)+1} - music") 
            else:
                print(f"{int(s/target_sr)} : {int(s/target_sr)+1} - speech")
                truncated_sample = sample[:, s:]
                #######################
                ### Enhancement
                #######################
                # 경로 및 파라미터들 직접 작성
                
                audio_dir_prediction = '/home/ubuntu/jupyter/minyoung/skt/Speech-enhancement/demo_data/test'
                dir_save_prediction = '/home/ubuntu/jupyter/minyoung/skt/Speech-enhancement/demo_data/test/sample_denoise/'
                audio_input_prediction = [truncated_sample]
                audio_output_prediction = 'sample32_denoise.wav'
                # Prediction 및 enhancement 단계
                prediction(weights_path, name_model, audio_dir_prediction, dir_save_prediction,
                            audio_input_prediction, audio_output_prediction, 8000, 1.0,
                            8064, 8064, 255, 63)

        else:
            print(f"{int(s/target_sr)} : {int(s/target_sr)+1} - {is_sys}") 

In [ ]:
adgadg

In [4]:
# from pydub import AudioSegment
# import os

# def split_audio(file_path, output_folder, split_length_ms=1000):
#     # 오디오 파일 로드
#     audio = AudioSegment.from_file(file_path)

#     # 전체 길이 계산 (밀리초 단위)
#     total_length_ms = len(audio)

#     # 출력을 저장할 폴더가 없다면 생성
#     if not os.path.exists(output_folder):
#         os.makedirs(output_folder)

#     # 1초 단위로 오디오 자르기
#     for i in range(0, total_length_ms, split_length_ms):
#         split_audio = audio[i:i+split_length_ms]
#         # 파일 저장 (0001, 0002 형식으로 저장)
#         split_filename = os.path.join(output_folder, f"split_{i//split_length_ms:04d}.wav")
#         split_audio.export(split_filename, format="wav")
#         print(f"저장됨: {split_filename}")

# # 예시 사용법
# for file_path in [test_sample2_path, test_sample3_path, test_sample4_path, test_sample5_path, test_sample6_path, test_sample7_path]:
#     os.makedirs(f"/home/ubuntu/jupyter/SKT/submit/samples/{os.path.basename(file_path)[:-4]}", exist_ok = True)
#     output_folder = f"/home/ubuntu/jupyter/SKT/submit/samples/{os.path.basename(file_path)[:-4]}"
#     split_audio(file_path, output_folder)


In [5]:
# from pydub import AudioSegment
# import os
# ring_path = "/home/ubuntu/jupyter/SKT/data/ring"
# ring_list = os.listdir(ring_path)

# def split_audio(file_path, output_folder, split_length_ms=1000):
#     # 오디오 파일 로드
#     audio = AudioSegment.from_file(file_path)

#     # 전체 길이 계산 (밀리초 단위)
#     total_length_ms = len(audio)

#     # 출력을 저장할 폴더가 없다면 생성
#     if not os.path.exists(output_folder):
#         os.makedirs(output_folder)

#     # 1초 단위로 오디오 자르기
#     for i in range(0, total_length_ms, split_length_ms):
#         split_audio = audio[i:i+split_length_ms]
#         # 파일 저장 (0001, 0002 형식으로 저장)
#         split_filename = os.path.join(output_folder, f"{os.path.basename(file_path)[:-4]}{i//split_length_ms:04d}.wav")
#         split_audio.export(split_filename, format="wav")
#         print(f"저장됨: {split_filename}")

# # 예시 사용법
# for file_path in ring_list:
#     output_folder = f"/home/ubuntu/jupyter/SKT/data/ring_split"
#     os.makedirs(output_folder, exist_ok = True)
#     split_audio(f"{ring_path}/{file_path}", output_folder)


In [6]:
# import os
# import pickle
# from tqdm.notebook import tqdm
# ring_path = f"/home/ubuntu/jupyter/SKT/data/ring_split" # path for system ringbacktone
# ring_list = [f"{ring_path}/{w}" for w in os.listdir(ring_path)] # 1초 이하 제거
# # ring2idx = {r:i for i, r in enumerate(ring_list)}
# # idx2ring = {i:os.path.basename(r) for i, r in enumerate(ring_list)}

# ring_specs = []
# idx2ring = {}
# i = 0
# for r in tqdm(ring_list):
#     ring_sec = load_and_resample(r, target_sr)
#     print(ring_sec.shape)
#     try:
#         ring_filter = preprocess(ring_sec, n_fft, hop_length, hann_window, target_sr, config['freq_range']).float()
#     except:
#         continue
#     spec = torch.zeros([513, 63])
#     spec[:,:ring_filter.shape[-1]] = ring_filter.squeeze()
#     ring_specs.append(spec)
#     idx2ring[i] = '_'.join(os.path.basename(r)[:-4].split('_')[:-1])
#     i+=1

# # np.save('ring_label2.npy', np.array(labels))
# np.save('ring_spec2.npy', torch.stack(ring_specs).numpy())
# with open('idx2ring2.pickle','wb') as fw:
#     pickle.dump(idx2ring, fw)

In [7]:
# from prediction_denoise import prediction
# import os
# import numpy as np
# import librosa
# import warnings
# warnings.filterwarnings('ignore')
# weights_path = '/home/ubuntu/jupyter/SKT/submit/weights'
# name_model = 'model_unet'


# sample_dir = "/home/ubuntu/jupyter/SKT/submit/samples"
# test_list = os.listdir(sample_dir)

# for test_folder in test_list:
#     print('\n',test_folder)

#     # 링백톤 분류기 불러오기
#     RING = CLSforRing()
#     # 1초 단위로 wav 파일 불러오기
#     ## 1초 단위 wav list
#     test_samples = sorted(os.listdir(f"{sample_dir}/{test_folder}"))
#     s = 0
#     for path in test_samples:
#         sample_sec = load_and_resample(f"{sample_dir}/{test_folder}/{path}", target_sr)
#         sample_filter = torch.empty([513, 63])
#         sample_filter_ = preprocess(sample_sec, n_fft, hop_length, hann_window, target_sr, config['freq_range']).float()
#         sample_filter[:,:sample_filter_.shape[-1]] = sample_filter_
#         is_sys = RING.cls(sample_filter.reshape(1,-1))
#         if is_sys == 'no sys':
#             with torch.no_grad():
#                 out_dict = model(forpred(sample_filter.unsqueeze(0)).unsqueeze(0).type(torch.FloatTensor).transpose(2,3).to(device))
#             test_pred=out_dict['event_logit']
#             no_speech = torch.argmax(test_pred, dim = -1).item()
#             if no_speech:
#                 print(f"{s} : {s+1} - music") 
#             else:
#                 print(f"{s} : {s+1} - speech")
#                 #######################
#                 ### Enhancement
#                 #######################
#                 # 경로 및 파라미터들 직접 작성
#                 audio_dir_prediction = f"{sample_dir}/{test_folder}/"
#                 dir_save_prediction = audio_dir_prediction.replace('samples','enhance')
#                 audio_input_prediction = [path]
#                 audio_output_prediction = path
#                 # Prediction 및 enhancement 단계
#                 prediction(weights_path, name_model, audio_dir_prediction, dir_save_prediction,
#                             audio_input_prediction, audio_output_prediction, 8000, 1.0,
#                             8064, 8064, 255, 63)
#         else:
#             print(f"{s} : {s+1} - {is_sys}") 
#         s += 1